# Bitcoin Price Prediction

##### Feito por: Henrique, Omar, Nevton

### Introdução
#### As criptomoedas estão cada vez mais ganhando espaço no mercado financeiro. Muitos dizem que as criptomoedas não passam de uma bolha especulativa, porém mesmo após uma forte queda a partir de 2018, o preço desses ativos em geral se mantem em um patamar relevante. Devido a alta volatilidade achamos que seria interessante fazer uma análise preditiva sobre o preço dos ativos uma vez que com grande volartilidade existem maiores oportunidades para se ganhar dinheiro. Escolhemos fazer apenas com Bitcoin, pois é a primeira, maior e principal criptomoeda.  
#### Dessa forma nosso objetivo era criar um modelo para tentar prever o preço do bitcoin, para a partir disso tomar decisões de investimento. Para fazer isso, primeiramente coletamos diversas variáveis sobre a rede do Bitcoin, nos ultimos 5 anos. Essas informações podem ser achadas no link: https://www.blockchain.com/charts. E relacionamos essas variáveis com nossa variável chave que era market_price2 (preço em dólares)
#### Nossa pergunta motriz foi: Qual vai ser o preço do Bitcoin No Futuro?

#### Para responder essa pergunta fizemos um regressão multipla e também usamos um método chamado Decision Tree Regressor

In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
dfs = pd.read_excel('DataFrame_bitcoin.xlsx')
data = pd.read_csv('difficulty.csv')

### Gráfico de como o preço do bitcoin se comportou de 2014 até agora

In [ ]:
plt.plot(dfs['market_price_2'])

### Análise exploratória
Primeiramente foi feito uma análise exploratória dos dados. Dados que estamos cruzando variáveis quantitativas o melhor método para vizualizarmos é com um scatter plot. Por termos um número grande de variáveis, primeiro fizemos uma tabela de correlação e analisamos, através do scatter plot, apenas as variáveis que mais se correlacionavam com nossa variável de interrese: 'Market_price_2'

In [ ]:
tabela_corr = dfs.corr()
tabela_corr

### Como podemos ver as variáveis que apresentam a correlação mais forte com o market price (preço) são:  
-miners_revenue  
-Dificuldade  
-Hash_rate<br>
-Market_Cap<br>
-Volume Estimado Da Transacao<br>
#### Agora vamos analisar mais a fundo essas relações

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["miners_revenue"])
plt.title('Preço x remuneração dos mineradores')
plt.xlabel('Preço')
plt.ylabel('remuneração dos mineradores')

Notasse uma forte correlação positiva

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["difficulty"])
plt.title('Preço x Dificulade')
plt.xlabel('Preço')
plt.ylabel('Dificuladade')

Podemos ver que quanto mais difícil é para minerar um bloco maior o preço

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["hash_rate"])
plt.title('Preço x Hash Rate')
plt.xlabel('Preço')
plt.ylabel('Hash Rate')

A relação é muito parecida com a descrita a cima. Ao fazer uma análise mais profunda sobre os conceitos intrincecos dessas duas variáveis, percebe-se que no fundo o hash rate e a dificuldade são quase a mesma coisa.  
Pode-se encontrar mais informações sobre esses conceitos em : https://2miners.com/blog/mining-difficulty-and-network-hashrate-explained/

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["market_cap"])
plt.title('Preço x Market_Cap')
plt.xlabel('Preço')
plt.ylabel('Market_Cap')

O market cap se mostrou muito relacionado com o preço, porém ao analisar o que a variável significa, concluimos que o market cap é um desdobramento do preço. Formula para se achar o market cap = Preço * quantidade de moedas em circulação. Por isso vamos excluir essa variável para fazer as regressões

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["estimated_transaction_volume_usd"])
plt.title('Preço x Volumne Estimado Da Transacao')
plt.xlabel('Preço')
plt.ylabel('Volumne Estimado Da Transacao')

Podemos observar uma correlacão forte positiva, ou seja, quanto maior o volume de transações maior o preço. Isso nos indica que se a rede esta mais ativa, mais pessoas querendo transacionar, o preco tende a aumentar

In [ ]:
df_treinamento = dfs.iloc[550:991]
df_teste = dfs.iloc[991:]

In [ ]:
Y_treinamento = df_treinamento['market_price_2']
X_treinamento = df_treinamento[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'miners_revenue',
 'n_transactions',
 'transaction_fees'
]]
Y_teste = df_teste['market_price_2']
X_teste = df_teste[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'miners_revenue',
 'n_transactions',
 'transaction_fees'
]]

## Fazendo a Regressão linear multipla

A regressão linear multipla é um conjunto de técnicas estatísticas a fim de construir modelos que possam descrever de maneira razoável relações entre várias variáveis explicativas de um determinado processo. Chamamos de Regressão linear multipla por usarmos varias variaveis.

Para fazer a regressão dividimos o dataframe em 2, sendo o primeiro para treinar o modelo e o segundo para verificar a acurrácia desse

In [ ]:
#Funcao Que Calcula A Regressao Linear Multipla
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [ ]:
#Funcao Que Calcula O Valor RMS
def rms(y,y_pred):
    return (sum((y-y_pred)**2)*1/len(y))**0.5

In [ ]:
regress(X_treinamento,Y_treinamento).summary()

#### Em nossa primeira regressão obtivemos um r-squared de 93.3%, ou seja, nosso modelo se ajusta aos pontos em 93.3% dos casos.

#### Neste primeiro modelo, no entanto estamos apenas usando os dados de um dia atrás, o que torna nosso modelo muito simples e falho. Dessa forma vamos varias a janela temporal usada para verificar qual a melhor janela de tempo para se usar

In [ ]:
n = len(Y_treinamento)
lista = []
dias = []
for i in range(20):
    Y_offset = Y_treinamento.values[i:]
    X_offset = X_treinamento.values[:(n-i),:]
    results = regress(X_offset,Y_offset)
    lista.append(results)
    dias.append(i)
    print(i, results.rsquared)

#### Decidimos fazer a analise usando 10 períodos anteriores, uma vez que analisando até 200 períodos esse foi com maior precisão, ainda sim usando dados anteriores

In [ ]:
i = 10
Y_offset1 = Y_treinamento.values[i:]
X_offset1 = X_treinamento.values[:(n-i),:]
modelo = regress(X_offset1,Y_offset1)
modelo.summary()

In [ ]:
X_test_c = sm.add_constant(X_teste1)
y_pred=modelo.predict(X_test_c)
print("R2  =",r2_score(Y_teste, y_pred))
print("RMS =",rms(Y_teste, y_pred))

In [ ]:
lista = np.linspace(0, 329, 329)
plt.title('Preço real e previsto')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred, label = 'Preço Previsto')
plt.legend()

Como podemos analisar no resultado da regressão tivemos algumas variáveis com um  valor p a cima do nível de significancia (0.05). Isso nos mostra que não é possível ter certeza que essas variáveis influenciam no modelo. No entanto, as varíveis que estão dentro desse nível nos podemos confirmar que são influentes no modelo

In [ ]:
X_treinamento1 = df_treinamento[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
]]
X_teste1 = df_teste[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
]]

In [ ]:
i = 10
Y_offset1 = Y_treinamento.values[i:]
X_offset1 = X_treinamento1.values[:(n-i),:]
modelo = regress(X_offset1,Y_offset1)
modelo.summary()

Ao tirar as variáveis é possível perceber que não houve melhora na performace do código e por isso usaremos os modelos com mais variáveis

#### Em nossa segunda regressão obtivemos um r-squared de 74.3%, ou seja, nosso modelo se ajusta aos pontos em 74.3% dos casos.

## Fazendo o  Decision Tree Regressor

Uma árvore de decisão é um modelo de machine learning supervisionada, usado para prever um destino, aprendendo regras de decisão a partir de recursos. Como o nome sugere, podemos pensar nesse modelo como quebrar nossos dados, tomando uma decisão com base em fazer uma série de perguntas. O modelo de árvore de decisão aprende uma série de perguntas para inferir os rótulos de classe das amostras.

In [ ]:
#Decision Tree Regressor
Arvore = DecisionTreeRegressor(random_state = 1)
Arvore.fit(X_treinamento, Y_treinamento)
plt.scatter(Y_teste, Arvore.predict(X_teste), color = 'blue')
plt.show()

In [ ]:
y_pred1=Arvore.predict(X_teste)
print("R2  =",r2_score(Y_teste, y_pred1))
print("RMS =",rms(Y_teste, y_pred1))

In [ ]:
plt.title('Preço real e previsto')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred1, label = 'Preço Previsto')
plt.legend()

## Fazendo o Random Forest Regressor

O metodo de regrassao 'Random Forest', se utiliza dos mesmo recursos do 'Decision Tree Regressor', gerando varias arvores de decisao e combinando os resultados, produz geralmente um resultado mais satisfatorio que a decision tree.

In [ ]:
RandomF = RandomForestRegressor(random_state=0)
RandomF.fit(X_treinamento1, Y_treinamento)
plt.scatter(Y_teste, RandomF.predict(X_teste1), color = 'blue')
plt.show()

In [ ]:
y_pred2=RandomF.predict(X_teste1)
print("R2  =",r2_score(Y_teste, y_pred2))
print("RMS =",rms(Y_teste, y_pred2))

In [ ]:
plt.title('Preço real e previsto')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred2, label = 'Preço Previsto')
plt.legend()

## Conclusão

In [ ]:
plt.title('Preço real e previsto usando regressão multipla')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred, label = 'Preço Previsto')
plt.legend()
plt.show()
plt.title('Preço real e previsto usando Decision Tree Regressor')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred, label = 'Preço Previsto')
plt.legend()
plt.show()
plt.title('Preço real e previsto usando Random Forest Regressor')
plt.plot(lista,Y_teste, label='Preço real')
plt.plot(lista,y_pred, label = 'Preço Previsto')
plt.legend()
plt.show()

Como podemos ver o método de previsão que chegou mais perto de prever o preço foi a regressão múltipla. Mesmo assim é possível perceber que todos os modelos são muito falhos e apresentam grande erro ao mostrar a realidade.
Ao nos depara com esse resultado tentamos fazer diversas iteração para tentar melhorar o modelo, porém todas se mostraram falhas.
Após fazer uma análise, concluimos algumas razões que podem explicar os resultados.  
-A dinamica do bitcoin mudou muito de 2014 pra cá, houve uma mudança de cenário e paradigma na cripto economia e, por isso, os dados do passado podem não se aplicar para explicar o presente  
-Por ser um mercado muito especulativo, as variáveis técnicas que dizem respeito a saúde da rede blockchain podem não ter tanta importância no seu valor final  
Pensamos que para próximas iteraões poderíamos fazer uso de um naive-bayes, do mesmo jeito que foi feito no projeto 2, para fazer uma análise sentimental em diversos portais de noticias e fóruns. Dessa forma poderíamos saber o que as pessoas estão achando e a partir disso concluir se o preço do ativo vai cair ou subir

## Referencias Bibliograficas

https://www.blockchain.com/charts

https://changelly.com/blog/bitcoin-price-prediction-2019-2025-2030/